# Motivation
论文原文：[《From Louvain to Leiden: guaranteeing well-connected communities》](https://arxiv.org/pdf/2012.07436)

参考学习地址：
* [AAAI21最佳论文Informer：效果远超Transformer的长序列预测神器！](https://zhuanlan.zhihu.com/p/349858834)
* [Informer: 一个基于Transformer的效率优化的长时间序列预测模型](https://zhuanlan.zhihu.com/p/355133560)
* [效果远超Transformer！AAAI 2021最佳论文Informer：最强最快的序列预测神器](https://zhuanlan.zhihu.com/p/349800169)

代码库：https://github.com/zhouhaoyi/Informer2020

Transformer在长序列时间序列预测有提高预测能力的潜力，但也存在一些问题：二次时间复杂度（$O(N^2)$）、高内存使用率以及encoder-decoder体系结构的固有限制，以及推理step-by-step效率低。Informer是基于Transformer的LSTF（Long Sequence Time-Series Forecasting）模型，居右三个显著特定：
* ProbSparse Self-Attention
* Self-Attention提取通过将级联层输入减半来突出控制注意，并有效地处理超长的输入序列
* 产生式decoder虽然概念上简单，但在一个正向操作中预测长时间序列，而不是一步一步地进行，这大大提高了长序列预测的推理速度

<img style="display: block; margin: 0 auto;" src="../../../assets/images/informer-1.png" width = "400" height = "400" alt="Informer" align=center />

## 问题定义
在固定size的窗口预测中，在时刻$t$的输入为$X^t=\{x_1^t,...,x_{L_x}^t|x_i^t\in R^{d_x}\}$，要预测对应的输出序列$Y^t=\{y_1^t,...,y_{L_y}^t|y_i^t\in R^{d_y}\}$，LSTF问题鼓励输出一个更长的输出，特征维度不再依赖于univariate例子（$d_y\ge 1$）。
* Encoder-Decoder框架：将输入表示$X^t$进行编码为一个隐藏状态表示$H^t=\{h_1^t,...,h_{L_h}^t\}$，并将输出表示$Y^t$解码。推理过程中dynamic decoding（step-by-step），Decoder从前一个状态$h_k^t$计算一个隐藏状态$h_{k+1}^t$以及第$k$步的输出，然后对$k+1$个序列进行预测$y_{k+1}^t$
* 输入表示：为了增强时间序列输入的全局位置上下文和局部时间上下文，给出了统一的输入表示

## ProbSparse Self-Attention
传统的self-attention主要由（query，key，value）组成，$A(Q,K,V)=Softmax(\frac{QK^T}{\sqrt{d}})V$，其中$Q\in R^{L_Q * d}, K\in R^{L_k * d}, V\in R^{L_v * d}$，第$i$个attention被定义为核平滑的概率形式：$A(q_i,K,V) = \sum_j \frac{k(q_i, k_j)}{\sum_l k(q_i, k_l)}v_j=E_{p(k_j|q_i)}[v_j]$，其中$p(k_j|q_i)=k(q_i,k_j)/\sum_l k(q_i,k_l), k(q_i,k_j)$作为不对称和函数$exp(q_ik_j^T/\sqrt{d})$。self-attention需要$O(L_QL_K)$的内存以及二次点积计算代价。

首先对典型自注意的学习模式定性评估，“稀疏性”self-attention得分形成长尾分布（即少数点积对主要注意有贡献，其他点积对可以忽略）。根据上式可知，第$i$个query的attention，被定义为概率$p(k_j|q_i)$和$v$的组合，支配性内积倾向于非均匀分布（若$p(k_j|q_i)=1/L_k$则attention变为了$V$的简单求和）。所以需要定义Query Sparsity评估，根据Kullback-Leibler Divergence（KL散度），第$i$个query sparsity评估为：
$$
\begin{align}
KL(q||p) = ln \sum_{l=1}^{L_k}e^{\frac{q_ik_l^T}{\sqrt{d}}} - \frac{1}{L_k}\sum_{j=1}^{L_k}\frac{q_ik_j^T}{\sqrt{d}} - lnL_k
\end{align}
$$
提出常数项，重定义第$i$个query sparsity评估为：
$$
\begin{align}
M(q_i,K) = ln \sum_{j=1}^{L_k}e^{\frac{q_ik_j^T}{\sqrt{d}}} - \frac{1}{L_k}\sum_{j=1}^{L_k}\frac{q_ik_j^T}{\sqrt{d}}
\end{align}
$$

第一项是$q_i$在所有keys的Log-Sum-Exp（LSE），第二项是arithmetic均值。定义ProbSparse Self-Attention为：
$$
\begin{align}
A(Q,K,V) = Softmax(\frac{\bar{Q}K^T}{\sqrt{d}})V
\end{align}
$$
其中$\bar{Q}$是和$q$相同size的稀疏矩阵，仅包含稀疏评估$M(q,M)下top-u的queries，由采样factor $c$所控制，另$u=c\cdot lnL_Q$。这样self-attention对每个query-key lookup仅需计算$O(lnL_Q)$的内积，内存使用包含$O(L_KlnL_Q)$，但计算$M(q_i,K)$时需要计算每队的dot-product，即$O(L_QL_K)$，同时LSE还会带来潜在的数值问题，受此影响，本文提出了query-sparsity评估的近似（原论文中有个不等式引理给出了上下界，主要是利用了凸函数特性）：
$$
\begin{align}
\bar{M}(q_i,K) = max_j \{\frac{q_ik_j^T}{\sqrt{d}}\} - \frac{1}{L_k}\sum_{j=1}^{L_k}\frac{q_ik_j^T}{\sqrt{d}}
\end{align}
$$
由此可将时间和空间复杂度控制到$O(LlnL)$

## Encoder-Decoder

<img style="display: block; margin: 0 auto;" src="../../../assets/images/informer-2.png" width = "900" height = "400" alt="Informer" align=center />
图中的stamp可以考虑分为local timestamp（position embeddings）以及global timestamp（week & month & holiday Embeddings）

### Encoder
Encoder被设计用来抽取鲁棒的长序列输入的long-range依赖，在第$t$个序列输入$X^t$被转为矩阵$X_{feed\_en}^t\in R^{L_x * d_{model}}$。

#### Self-attention Distilling
作为ProbSparse Self-attention的自然结果，encoder的特征映射会带来值的冗余组合，利用distilling对具有支配特征的优势特征进行特权化，在下一层生成focus self-attention特征映射。它对输入的时间维度进行了锐利的修剪，$n$个头部权重（重叠的红色方块）。受扩展卷积的启发，distilling过程从第$j+1$层推进：
$$
\begin{align}
X_{j+1}^t = MaxPool(ELU(Conv1d([X_j^t]_{AB})))
\end{align}
$$
其中$[\cdot]_{AB}$包含Multi-Head ProbSparse self-attention以及重要的attention block操作。为了增强distilling操作的鲁棒性，构建了halving replicas，并通过一次删除一层（如上图）来逐步减少自关注提取层的数量，从而使它们的输出维度对齐。因此，我们将所有堆栈的输出串联起来，并得到encoder的最终隐藏表示。

### Decoder
此处使用标准的decoder结构，由2个一样的multihead attention层，但是，生成的inference被用来缓解速度瓶颈，我们使用下面的向量喂入decoder：
$$
\begin{align}
X_{feed\_de}^t = Concat(X_{token}^t, X_0^t) \in R^{(L_{token}+L_y)*d_{model}}
\end{align}
$$
其中，$X_{token}^t\in R^{L_{token}*d_{model}}$是start token，$X_0^t\in R^{L_y*d_{model}}$是输出sequence的占位符。<font color="red">将Masked multi-head attention应用于ProbSparse self-attention，将mask的点积设置为-inf</font>。可以防止每个位置都关注未来的位置。

#### Generative Inference
从长序列中采样一个$L_{token}$，这是在输出序列之前的slice。以预测168个点为例（7天温度预测7*24），将目标序列已知的前5天的值作为start token，并将$X_{feed\_de}={X_{5d}, X_0}$输入生成式推理解码器。$X_0$包含目标序列的时间戳，即目标周的上下文。注意，decoder通过一个前向过程预测所有输出，并且不存在耗时的dynamic decoding。

### Loss Function
模型最终使用MSE损失作为最终的Loss